# Run Registry Data
This notebook showcases the use of the Run Registry api, `runregistry`, and serves as an example of how to use it. Read the documentation found in the `runregistry_api_client` GitHub repository for more information on how to authenticate: https://github.com/cms-DQM/runregistry_api_client

In [ ]:
import runregistry
import itertools
runregistry.setup("production")

In [ ]:
# Getting run registry data
filter = {
    # "run_number": {
        # "and": [{">=": 365753}, {"<=": 373710}]
        # "and": [{"=": 365753}]
    # },
    "class": {
        "and": [{"=": "Collisions23"}]
    },
    "tracker-strip": "GOOD",
    "tracker-pixel": "GOOD",
}

runs = runregistry.get_runs(filter=filter)
runs

In [ ]:
# Generating json
json_logic = {
  "and": [
    {"==": [{"var": "lumisection.rr.tracker-pixel"}, "GOOD"        ]},
    {"==": [{"var": "lumisection.rr.tracker-strip"}, "GOOD"        ]},
    {"==": [{"var": "lumisection.rr.tracker-track"}, "GOOD"        ]},
  ]
}

generated_jsons = {}
datasets = ['/Express/Collisions{}/DQM', '/PromptReco/Collisions{}/DQM']
years = ['2022', '2023', '2024']
for dataset, year in itertools.product(datasets, years):
    dataset_name = dataset.format(year)
    print(dataset_name) 
    generated_jsons[dataset_name] = {
        **runregistry.create_json(
            json_logic=json_logic, 
            dataset_name_filter=dataset.format(year)
        )['generated_json']
    }

In [ ]:
generated_jsons.keys()

In [ ]:
jsonruns = {}
for generated_json in generated_jsons:
    jsonruns = {**jsonruns, **generated_json["generated_json"]}
len(jsonruns)

In [ ]:
jsonruns

In [ ]:
import pandas as pd
pd.DataFrame(list(jsonruns.items()), columns=None).convert_dtypes()

In [ ]:
import json
import os

json_fname = "Express-Collisions-2022-2023-2024_pixel-strip-track-good.json"
outfile_path = "/eos/home-i02/r/rcruzcan/SWAN_projects/RefRunRank/RefRunRank/jsons/"

with open (os.path.join(outfile_path, json_fname), "w") as outfile:
    json.dump(jsonruns, outfile)